# 3.1 Imports

In [9]:
# import libraries
import numpy as np
import pandas as pd

import urllib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import time
from PIL import Image

from sklearn.metrics.pairwise import pairwise_distances

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [10]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [11]:
# create function
# open web browser

def configure_driver():
    # incognito window
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    
    # add the argument and make the browser Headless.
    # chrome_options.add_argument("--headless")

    # open web browser
    driver = webdriver.Chrome('/Users/abrahamleung/Documents/chromedriver', options=chrome_options)
    return driver

In [12]:
# list of categories
categories = [
    'activewear',
    'jackets',
    'sweatshirts-hoodies'
]

In [24]:
# load data
df = pd.read_csv('data/activewear.csv')

In [25]:
df.head()

,name,price,img_file,url,num_price,color,apparel,key_words
0,CK ONE TANGA BIKINI,HKD 392.00,image/CK ONE TANGA BIKINI.png,https://www.calvinklein.com/hk/en/ck-one-tanga...,392.0,BLACK CUT OUT PRINT,BIKINI,BLACK CUT OUT PRINT BIKINI
1,CK ONE BRALETTE,HKD 392.00,image/CK ONE BRALETTE.png,https://www.calvinklein.com/hk/en/ck-one-brale...,392.0,BLACK CUT OUT PRINT,BRALETTE,BLACK CUT OUT PRINT BRALETTE
2,ACTIVE ICON WOVEN SHORTS,HKD 413.00,image/ACTIVE ICON WOVEN SHORTS.png,https://www.calvinklein.com/hk/en/active-icon-...,413.0,RASPBERRY BLUSH,SHORTS,RASPBERRY BLUSH SHORTS
3,ACTIVE ICON FULL LENGTH ALL-OVER PRINT LEGGINGS,HKD 483.00,image/ACTIVE ICON FULL LENGTH ALL-OVER PRINT L...,https://www.calvinklein.com/hk/en/active-icon-...,483.0,CK BLK CK AOP,LEGGINGS,CK BLK CK AOP LEGGINGS
4,LACE LIGHTWEAR JACKET,HKD 903.00,image/LACE LIGHTWEAR JACKET.png,https://www.calvinklein.com/hk/en/lace-lightwe...,903.0,BRIGHT WHITE,JACKET,BRIGHT WHITE JACKET


In [26]:
len(df)

12

# 3.2 Amazon Search Engine Setup

In [11]:
# open web browser
driver = configure_driver()

In [12]:
# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

In [13]:
# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

In [18]:
# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath('//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]')
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

In [19]:
# select 'women's fashion' as department
department = driver.find_element_by_xpath('//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]')
department.click()

# may need time sleep
time.sleep(0.1)

In [6]:
# (unsuccessful) create function
# driver cannot serve as a variable
# it means the driver cannot be input nor output through a function
# Amazon search engine setup

def amazon_setup():

    # open web browser
    driver = configure_driver()

    # search engine
    search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

    # input website
    driver.get(search_engine)

    # may need time sleep
    time.sleep(0.5)

    # Choose HKD

    # select option
    option = driver.find_element_by_id('icp-touch-link-cop')
    option.click()

    # may need time sleep
    time.sleep(0.5)

    # select currency setting
    currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
    currency.click()

    # select hkd as currency
    hkd = driver.find_element_by_xpath(
        '//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]'
    )
    hkd.click()

    # save change
    save = driver.find_element_by_id('icp-btn-save')
    save.click()

    # may need time sleep
    time.sleep(0.5)

    # select 'women's fashion' as department
    department = driver.find_element_by_xpath(
        '//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]'
    )
    department.click()

    # may need time sleep
    time.sleep(0.1)
    
    return driver

# 3.3 Scraping of Every Link

In [31]:
# testing
n = 0

key_words = df['key_words'].iloc[n]

In [33]:
key_words

'BLACK CUT OUT PRINT BIKINI'

In [21]:
# input search item

search_bar = driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys(key_words)
search_bar.send_keys(Keys.ENTER)

# may need time sleep
time.sleep(0.5)

In [22]:
# get links

links = []
link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
for link in link_paths:
    x = link.get_attribute('href')
    links.append(x)

In [23]:
# no of links can be more than 50
len(links)

48

In [24]:
links[:5]

['https://www.amazon.com/CUPSHE-Womens-Floral-Halter-Swimsuit/dp/B07S223TYL/ref=sr_1_1?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-1',
 'https://www.amazon.com/EJsoyo-Snakeskin-Swimwear-Brazilian-Swimsuits/dp/B0918HCKQT/ref=sr_1_2?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-2',
 'https://www.amazon.com/SHEKINI-Spaghetti-Strappy-Swimsuit-Manhattan/dp/B075P8JS6N/ref=sr_1_3?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-3',
 'https://www.amazon.com/Nulibenna-Swimsuits-Waisted-Swimwear-Bathing/dp/B083J6VHTJ/ref=sr_1_4?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-4',
 'https://www.amazon.com/CUPSHE-Womens-Swimsuit-Strappy-Bathing/dp/B08N146G6S/ref=sr_1_5?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-5']

In [11]:
# create function
# scraping every link

def all_links(driver, key_words):

    # input search item
    search_bar = driver.find_element_by_id('twotabsearchtextbox')
    search_bar.send_keys(key_words)
    search_bar.send_keys(Keys.ENTER)

    # may need time sleep
    time.sleep(0.5)

    # get links
    links = []
    link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
    for link in link_paths:
        x = link.get_attribute('href')
        links.append(x)
    
    return links

# 3.4 Scraping of Search Result Details

In [27]:
# testing
# url

URL = links[0]
URL

'https://www.amazon.com/CUPSHE-Womens-Floral-Halter-Swimsuit/dp/B07S223TYL/ref=sr_1_1?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-1'

In [28]:
# input website
driver.get(URL)

In [29]:
# product names
try:
    name = driver.find_elements_by_tag_name('h1')
    name = name[0].text
except:
    name = None

In [30]:
name

"CUPSHE Women's One Piece Swimsuit Cutout Halter Lace Up Twist Bathing Suit"

In [31]:
# product price
try:
    price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
    price = price[0].text
except:
    price = None

In [32]:
price

'HKD 237.32'

In [39]:
# testing
index = 0

In [40]:
# product image
try:
    # size
    img_width,img_height = 300,300

    # get image
    img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
    img = img[0]

    # 'src' = get image source
    src = img.get_attribute('src')

    # download image
    urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

    # resize image (smaller size)
    ori_img = Image.open(f'image/Amazon/{index}.png')
    resize_img = ori_img.resize((img_width,img_height))
    resize_img.save(f'image/Amazon/{index}.png')

    img_file = f'image/Amazon/{index}.png'
except:
    img_file = None


In [41]:
img_file

'image/Amazon/0.png'

In [8]:
# create function
# search result details

def scrape_details(index, link):

    # input website
    driver.get(link)

    # product names
    try:
        name = driver.find_elements_by_tag_name('h1')
        name = name[0].text
    except:
        name = None

    # product price
    try:
        price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
        price = price[0].text
    except:
        price = None
    
    # product image
    try:
        # size
        img_width,img_height = 300,300

        # get image
        img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
        img = img[0]

        # 'src' = get image source
        src = img.get_attribute('src')

        # download image
        urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

        # resize image (smaller size)
        ori_img = Image.open(f'image/Amazon/{index}.png')
        resize_img = ori_img.resize((img_width,img_height))
        resize_img.save(f'image/Amazon/{index}.png')

        img_file = f'image/Amazon/{index}.png'
    except:
        img_file = None

    return name, price, img_file

In [9]:
# scrape every link

names = []
prices = []
img_files = []
index = 0

for link in links:
    
    # scrape name, price and img_file
    name, price, img_file = scrape_details(index, link)

    # append to lists
    names.append(name)
    prices.append(price)
    img_files.append(img_file)

    # update index
    index += 1

# create dataframe

df = pd.DataFrame({
    'name': names,
    'price': prices,
    'img_file': img_files,
    'url': links
})

NameError: name 'links' is not defined

# 3.5 Product Recommendations

In [13]:
# input image shape
img_width, img_height, _ = 224, 224, 3

In [14]:
# load model
# as no model training required, set 'compile=False'

model = load_model('cnn_model.h5', compile=False)

In [15]:
# create function
# feature extraction for image

def get_embedding(model, img_file):
    # reshape
    img = image.load_img(img_file, target_size=(img_width, img_height))

    # img to numpy array
    x = image.img_to_array(img)

    # expand dim (1, w, h)
    x = np.expand_dims(x, axis=0)

    # Preprocess input
    x = preprocess_input(x)

    return model.predict(x).reshape(-1)

In [68]:
# Function that get product recommendations based on the cosine similarity score of product image

def get_recommender(idx, df, top_n):

    # select the target image (default is index 0)
    target_index = indices[idx]

    # calculate the similarity scores
    similarity_scores = list(enumerate(similarity[target_index]))

    # sort the result 
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # select the top n items
    similarity_scores = similarity_scores[1:top_n+1]

    # select the indices of top n items
    top_indices = [i[0] for i in similarity_scores]

    # idx_sim = [i[1] for i in sim_scores]

    return indices.iloc[top_indices].index

In [50]:
# testing CK image

target_name = ['CK ONE TANGA BIKINI']
target_img_file = ['image/CK ONE TANGA BIKINI.png']

df_target = pd.DataFrame({
    'img_file': target_img_file
})

In [51]:
df_target

,img_file
0,image/CK ONE TANGA BIKINI.png


In [52]:
# create df for testing

img_files = []
for i in range(48):
    img_files.append(f'image/Amazon/{i}.png')

# create df
df_amazon = pd.DataFrame({
    'img_file': img_files
})

In [55]:
# append amazon dataframe to the target dataframe

df = df_target.append(df_amazon, ignore_index=True)

In [57]:
len(df)

49

In [56]:
df.head()

,img_file
0,image/CK ONE TANGA BIKINI.png
1,image/Amazon/0.png
2,image/Amazon/1.png
3,image/Amazon/2.png
4,image/Amazon/3.png


In [24]:
# get item embedding with corresponding image
# image_embedding = get_embedding(model, target_img_file)

In [58]:
# make a dataframe duplicate
df_copy = df

# get image embeddings for each image
map_embeddings = df_copy['img_file'].apply(lambda img: get_embedding(model, img))

# convert series of lists to dataframe
df_embeddings = map_embeddings.apply(pd.Series)

In [59]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,16.656422,1.759933,0.396486,5.663714,0.000000,1.727282,0.442207,0.000000,0.000000,...,0.000000,0.000000,2.258237,2.662622,3.287225,0.0,12.032290,3.004779,0.000000,1.093109
1,0.000000,7.160003,1.574659,4.338456,2.551096,2.673633,4.204260,6.550857,0.942263,2.079585,...,0.975850,0.997066,2.406110,25.856796,3.466363,0.0,7.697368,0.309879,2.489410,7.677667
2,1.831643,22.057829,2.530438,1.242392,0.000000,0.000000,1.320725,3.649347,0.030288,0.000000,...,7.023321,2.911746,0.000000,1.198196,2.623420,0.0,2.178410,1.167908,3.153160,12.983468
3,3.890997,2.239640,0.561372,6.952099,2.865389,3.875372,6.084309,2.111632,2.495424,4.386785,...,0.406559,2.324595,1.143666,17.603485,2.035505,0.0,1.155617,2.491142,0.000000,5.401934
4,0.479622,3.835210,3.169219,5.755448,1.441075,5.746672,3.464339,4.615533,4.365491,2.138535,...,12.700304,0.725670,2.027468,11.041115,0.000000,0.0,0.997130,1.629200,1.458257,10.393693


In [60]:
## compute similarity between items
# calculate distance matrix
similarity = 1 - pairwise_distances(df_embeddings, metric='cosine')

In [64]:
similarity[0]

array([1.        , 0.5165263 , 0.5369592 , 0.51267225, 0.5224726 ,
       0.5031241 , 0.5237402 , 0.51745456, 0.5408669 , 0.47479218,
       0.48267484, 0.46357816, 0.50001   , 0.5165371 , 0.5172481 ,
       0.58180815, 0.49073946, 0.5257547 , 0.5269011 , 0.5430988 ,
       0.55302197, 0.57794875, 0.5942491 , 0.49452055, 0.5181655 ,
       0.5339118 , 0.552034  , 0.5226602 , 0.5101911 , 0.49412894,
       0.52358085, 0.5315855 , 0.51819026, 0.4978131 , 0.5014058 ,
       0.47400868, 0.5076667 , 0.5177233 , 0.494326  , 0.48988938,
       0.50520056, 0.5170815 , 0.49531442, 0.5356687 , 0.52722985,
       0.5314058 , 0.5180114 , 0.50884783, 0.4995228 ], dtype=float32)

In [65]:
# create dataframe for indices
indices = pd.Series(range(len(df)), index=df.index)

In [77]:
i = 14

img_name = df.iloc[0]['img_file'][6:-4] + str(i) + '.png'
img_name

'CK ONE TANGA BIKINI14.png'

In [69]:
# compute recommendations' indices
recommend_indices = get_recommender(0, df, 2)

# create lists
recommend_names = []
recommend_prices = []
recommend_img_files = []
recommend_urls = []

for i in recommend_indices:
    # recommend_names.append(df.iloc[i]['name'])
    # recommend_prices.append(df.iloc[i]['price'])
    # recommend_urls.append(df.iloc[i]['url'])

    # save the img_files separately
    # use 'i-1' because the target item was appended at the front of dataframe
    # the corresponding image index has to minus 1
    img = Image.open(f'image/Amazon/{i-1}.png')

    img_name = df.iloc[0]['img_file'][6:-4] + str(i) + '.png'
    img.save(f'image/recommendations/{img_name}.png')

    img_file = f'image/recommendations/{img_name}.png'

    # append img_file's new path
    recommend_img_files.append(img_file)

In [70]:
recommend_img_files

['image/Amazon/21.png', 'image/Amazon/14.png']

In [ ]:
# get item embedding with corresponding image
image_embedding = get_embedding(model, target_img_file)

# get embedding for all items in dataset
df_sample = df
map_embeddings = df_sample['img_file'].apply(lambda img: get_embedding(model, img))
df_embs = map_embeddings.apply(pd.Series)

## compute similarity between items
# calculate distance matrix
cosine_sim = 1 - pairwise_distances(df_embs, metric='cosine')

# Recommender Similar Items
indices = pd.Series(range(len(df)), index=df.index)

# Idx Item to Recommender
idx_ref = 2

try:
    # Recommendations
    idx_rec= get_recommender(idx_ref, df, 2)

    # create lists
    rec_names = []
    rec_prices = []
    rec_urls = []

    for i in idx_rec:
        rec_names.append(df.iloc[i]['name'])
        rec_prices.append(df.iloc[i]['unit_price'])
        rec_urls.append(df.iloc[i]['url'])
except:
    rec_names, rec_prices, rec_urls = None, None, None

# 3.6 Apply to All Categories

# 3.6.1 Activewear

In [51]:
# Amazon search engine setup

# open web browser
driver = configure_driver()

# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath(
    '//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]'
)
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

# select 'women's fashion' as department
department = driver.find_element_by_xpath(
    '//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]'
)
department.click()

# may need time sleep
time.sleep(0.1)

In [52]:
# activewear
category = 'activewear'
    
# load data
df = pd.read_csv(f'data/{category}.csv')

list_key_words = []

## scraping all links
for key_words in df['key_words'][:1]:
    
    start_time = time.time()

    # input website
    URL = f'https://www.amazon.com/s?k={key_words}&i=fashion-womens-intl-ship&ref=nb_sb_noss_2'
    driver.get(URL)

    # may need time sleep
    time.sleep(0.5)

    # get links
    links = []
    link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
    for link in link_paths:
        x = link.get_attribute('href')
        links.append(x)
    
    ## scrape every product details

    names = []
    prices = []
    img_files = []
    index = 0

    for link in links:
        
        # input website
        driver.get(link)

        # product names
        try:
            name = driver.find_elements_by_tag_name('h1')
            name = name[0].text
        except:
            name = None

        # product price
        try:
            price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
            price = price[0].text
        except:
            price = None
        
        # product image
        try:
            # size
            img_width,img_height = 300,300

            # get image
            img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
            img = img[0]

            # 'src' = get image source
            src = img.get_attribute('src')

            # download image
            urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

            # resize image (smaller size)
            ori_img = Image.open(f'image/Amazon/{index}.png')
            resize_img = ori_img.resize((img_width,img_height))
            resize_img.save(f'image/Amazon/{index}.png')

            img_file = f'image/Amazon/{index}.png'
        except:
            img_file = None

        # append to lists
        names.append(name)
        prices.append(price)
        img_files.append(img_file)

        # update index
        index += 1

    # create dataframe
    df = pd.DataFrame({
        'name': names,
        'price': prices,
        'img_file': img_files,
        'url': links
    })

    # remove row with missing values
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    ## recommender

    ## output lists for creating dataframe for final deployment use
    
    end_time = time.time()
    print(f'execution time: {end_time - start_time}s')

execution time: 152.3548548221588s
execution time: 146.66727685928345s


In [53]:
df

,name,price,img_file,url
0,SheIn Women's Criss Cross Back Lace Bra Deep V...,HKD 134.04,image/Amazon/0.png,https://www.amazon.com/SheIn-Womens-Scalloped-...
1,SweatyRocks Women's Summer Short Sleeve Tee Di...,HKD 78.81 - HKD 134.04,image/Amazon/1.png,https://www.amazon.com/SweatyRocks-Womens-Summ...
2,Verdusa Women's Sleeveless Scoop Neck Strappy ...,HKD 70.92 - HKD 149.81,image/Amazon/2.png,https://www.amazon.com/Verdusa-Sleeveless-Stra...
3,SHEKINI Women's Floral Print Swim Bottom Cutou...,HKD 188.94 - HKD 212.61,image/Amazon/3.png,https://www.amazon.com/SHEKINI-Womens-Spaghett...
4,RELLECIGA Women's Strappy Longline Triangle Bi...,HKD 89.15 - HKD 173.48,image/Amazon/4.png,https://www.amazon.com/RELLECIGA-Womens-Strapp...
5,NIKIBIKI Women Seamless One Shoulder Ribbed Br...,HKD 149.10 + No Import Fees Deposit & HKD 61.7...,image/Amazon/5.png,https://www.amazon.com/NIKIBIKI-Seamless-Shoul...
6,Womens Short Sleeve Graphic Tees Distressed Ha...,HKD 126.15 - HKD 173.48,image/Amazon/6.png,https://www.amazon.com/Womens-Distressed-Graph...
7,Ekaliy Women's 2 Piece Multi Wear Outfits Sexy...,HKD 212.85 - HKD 252.29,image/Amazon/7.png,https://www.amazon.com/Ekaliy-Outfits-Halter-B...
8,SheIn Women's Strappy Backless Summer Evening ...,HKD 260.26,image/Amazon/8.png,https://www.amazon.com/SheIn-Womens-Strappy-Ba...
9,RUNNING GIRL One Shoulder Sports Bra Removable...,HKD 126.15 - HKD 165.59,image/Amazon/9.png,https://www.amazon.com/RUNNING-GIRL-Shoulder-W...


In [1]:
df

NameError: name 'df' is not defined

In [ ]:
# testing log

# only scrape all links:
# 3.5s each

# scrape all links and scrape each product details:
# 2min 30s each

# 3.6.2 Jackets

# 3.6.3 Sweatshirts-Hoodies

In [110]:
# close web browser
driver.close()

In [ ]:
# call the final function as 'main'

In [ ]:
# check if target items displayed by WebDriverWait()
# to save time
try:
    WebDriverWait(driver, 5).until(lambda s: s.find_element_by_id("search-results-category-target").is_displayed())
except TimeoutException:
    return None

In [ ]:
# END